In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

NOT : Projedeki grafikler, "plotly" kullanıldığından kaynaklı "github" üzerinde gözükmeyebilir. Grafikleri de görebilmeniz için projeyi indirip kurmanızı tavsiye ederim.

#### 1 - Veri ekleme ve düzenleme

In [2]:
df = pd.read_excel('Çalışma Çizelgesi v2.xlsx')
df.tail(3)

,Gün,Ay,Yıl,GünAdı,Yapılan,Hedef,ÇalışmaOranı,FreeDay,OffDay
1028,28,9,2025,Pazar,0,0,0.000,1,0
1029,29,9,2025,Pazartesi,3,8,0.375,0,0
1030,30,9,2025,Salı,2,8,0.250,0,0


In [3]:
# Hedef belirlenen ancak çalışma yapılmayan günleri işaretleme
df['ÇalışmaOranı'] = np.round( ( df['ÇalışmaOranı'] * 100 ) , 1 )
df['Yapilmayan'] = [ 1 if a > 0 and b == 0 else 0 for a,b in zip(df['Hedef'] , df['Yapılan'])]

In [4]:
# Tarih sütunu düzenleme
df['Tarih'] = pd.to_datetime(df['Yıl'].astype(str) + '-' + df['Ay'].astype(str) + '-' + df['Gün'].astype(str))
df['Tarih'] = df['Tarih'].dt.strftime('%d-%m-%Y')
df.drop(columns=['Gün', 'Ay','Yıl'], inplace=True)
df = df[['Tarih', 'GünAdı', 'Yapılan', 'Hedef', 'ÇalışmaOranı', 'FreeDay', 'OffDay', 'Yapilmayan']]

#### 2 - Günlük Çalışmalar

##### Günlük TOPLAM Çalışmalar

In [5]:
day_count = df['GünAdı'].value_counts().reset_index()
day_count.columns = ['GünAdı', 'GünSayı']

toplam_calisma = df.groupby('GünAdı')[['Yapılan','Hedef', 'FreeDay', 'OffDay', 'Yapilmayan']].sum().reindex(['Pazartesi', 'Salı', 'Çarşamba', 'Perşembe', 'Cuma', 'Cumartesi', 'Pazar']).reset_index()
toplam_calisma['Çalışma %'] = np.round( ((toplam_calisma['Yapılan'] / toplam_calisma['Hedef']) * 100) ,1) 
toplam_calisma['BoşGünler'] = toplam_calisma['FreeDay'] + toplam_calisma['OffDay'] + toplam_calisma['Yapilmayan']
toplam_calisma = toplam_calisma[['GünAdı', 'Yapılan', 'Hedef', 'Çalışma %', 'FreeDay', 'OffDay', 'Yapilmayan', 'BoşGünler']]
toplam_calisma['Boş %'] = np.round( ((toplam_calisma['BoşGünler'] / df.shape[0]) * 100) ,1) 

toplam_calisma = toplam_calisma.merge(day_count, on='GünAdı', how='left')
toplam_calisma

,GünAdı,Yapılan,Hedef,Çalışma %,FreeDay,OffDay,Yapilmayan,BoşGünler,Boş %,GünSayı
0,Pazartesi,673,1192,56.5,2,10,6,18,1.7,148
1,Salı,530,1122,47.2,3,13,7,23,2.2,148
2,Çarşamba,513,1128,45.5,2,18,10,30,2.9,147
3,Perşembe,478,1040,46.0,2,24,9,35,3.4,147
4,Cuma,389,984,39.5,3,31,19,53,5.1,147
5,Cumartesi,94,194,48.5,99,14,4,117,11.3,147
6,Pazar,74,197,37.6,103,13,7,123,11.9,147


##### Günlük ORTALAMA Çalışmalar

In [6]:
ortalama_gunluk = df.groupby('GünAdı')[['Yapılan','Hedef', 'FreeDay', 'OffDay', 'Yapilmayan']].mean().reindex(['Pazartesi', 'Salı', 'Çarşamba', 'Perşembe', 'Cuma', 'Cumartesi', 'Pazar']).reset_index()
ortalama_gunluk['Çalışma %'] = np.round( ((ortalama_gunluk['Yapılan'] / ortalama_gunluk['Hedef']) * 100) ,1) 
ortalama_gunluk = ortalama_gunluk.round(1)
ortalama_gunluk

,GünAdı,Yapılan,Hedef,FreeDay,OffDay,Yapilmayan,Çalışma %
0,Pazartesi,4.5,8.1,0.0,0.1,0.0,56.5
1,Salı,3.6,7.6,0.0,0.1,0.0,47.2
2,Çarşamba,3.5,7.7,0.0,0.1,0.1,45.5
3,Perşembe,3.3,7.1,0.0,0.2,0.1,46.0
4,Cuma,2.6,6.7,0.0,0.2,0.1,39.5
5,Cumartesi,0.6,1.3,0.7,0.1,0.0,48.5
6,Pazar,0.5,1.3,0.7,0.1,0.0,37.6


#### 3 - Toplam ve Ortalama Çalışma Raporu

In [7]:
df_all = df.agg({'Yapılan': 'sum', 'Hedef': 'sum', 'FreeDay': 'sum', 'OffDay': 'sum', 'Yapilmayan': 'sum'})
df_all['Çalışma %'] = np.round((df_all['Yapılan'] / df_all['Hedef'])*100,1)
yapilmayan_toplam = df_all['FreeDay'] + df_all['OffDay'] + df_all['Yapilmayan']
df_all['BoşToplam'] = yapilmayan_toplam
df_all['Boş %'] = np.round((yapilmayan_toplam / df.shape[0])*100,1)
df_all

Yapılan       2751.0
Hedef         5857.0
FreeDay        214.0
OffDay         123.0
Yapilmayan      62.0
Çalışma %       47.0
BoşToplam      399.0
Boş %           38.7
dtype: float64

In [8]:
print(f"Toplam Çalışma Yapılan Gün Sayısı: {df.shape[0]}")

Toplam Çalışma Yapılan Gün Sayısı: 1031


In [9]:
df_ort = df.agg({'Yapılan': 'mean', 'Hedef': 'mean', 'FreeDay': 'mean', 'OffDay': 'mean', 'Yapilmayan': 'mean'})
df_ort.round(1)

Yapılan       2.7
Hedef         5.7
FreeDay       0.2
OffDay        0.1
Yapilmayan    0.1
dtype: float64

#### 4 - Aylık Çalışmalar

In [10]:
df['Tarih'] = pd.to_datetime(df['Tarih'], format='%d-%m-%Y')
aylik_toplam = df.groupby(df['Tarih'].dt.to_period("M")).agg({'Yapılan': 'sum', 'Hedef': 'sum', 'FreeDay': 'sum', 'OffDay': 'sum', 'Yapilmayan': 'sum'}).reset_index()
aylik_toplam['Gün'] = df.groupby(df['Tarih'].dt.to_period("M")).size().values

aylik_toplam['ÇalışmaOranı'] = aylik_toplam['Yapılan'] / aylik_toplam['Hedef'] 
aylik_toplam['ÇalışmaOranı'] = np.round(aylik_toplam['ÇalışmaOranı'], 3) * 100

aylik_toplam['BoşGünler'] = aylik_toplam['FreeDay'] + aylik_toplam['OffDay'] + aylik_toplam['Yapilmayan']

aylik_toplam['Boş %'] = np.round((aylik_toplam['BoşGünler']/aylik_toplam['Gün'])*100 , 1)

aylik_toplam= aylik_toplam[['Tarih','Gün','Yapılan', 'Hedef', 'ÇalışmaOranı','FreeDay','OffDay', 'Yapilmayan', 'BoşGünler','Boş %']]
aylik_toplam

,Tarih,Gün,Yapılan,Hedef,ÇalışmaOranı,FreeDay,OffDay,Yapilmayan,BoşGünler,Boş %
0,2022-12,27,59,124,47.6,0,4,6,10,37.0
1,2023-01,31,101,148,68.2,7,2,1,10,32.3
2,2023-02,28,43,76,56.6,6,13,2,21,75.0
3,2023-03,31,107,172,62.2,7,3,1,11,35.5
4,2023-04,30,76,178,42.7,5,5,4,14,46.7
5,2023-05,31,116,224,51.8,5,2,0,7,22.6
6,2023-06,30,127,216,58.8,0,8,0,8,26.7
7,2023-07,31,34,88,38.6,1,7,9,17,54.8
8,2023-08,31,9,50,18.0,2,16,9,27,87.1
9,2023-09,30,18,44,40.9,5,14,2,21,70.0


##### Aylık Ortalama

In [11]:
aylik_ort = df.groupby(df['Tarih'].dt.to_period("M")).agg({'Yapılan': 'mean', 'Hedef': 'mean', 'FreeDay': 'mean', 'OffDay': 'mean', 'Yapilmayan': 'mean'}).reset_index()
aylik_ort['BoşGünler'] = aylik_ort['FreeDay'] + aylik_ort['OffDay'] + aylik_ort['Yapilmayan']
aylik_ort = aylik_ort.round(1)
aylik_ort

,Tarih,Yapılan,Hedef,FreeDay,OffDay,Yapilmayan,BoşGünler
0,2022-12,2.2,4.6,0.0,0.1,0.2,0.4
1,2023-01,3.3,4.8,0.2,0.1,0.0,0.3
2,2023-02,1.5,2.7,0.2,0.5,0.1,0.8
3,2023-03,3.5,5.5,0.2,0.1,0.0,0.4
4,2023-04,2.5,5.9,0.2,0.2,0.1,0.5
5,2023-05,3.7,7.2,0.2,0.1,0.0,0.2
6,2023-06,4.2,7.2,0.0,0.3,0.0,0.3
7,2023-07,1.1,2.8,0.0,0.2,0.3,0.5
8,2023-08,0.3,1.6,0.1,0.5,0.3,0.9
9,2023-09,0.6,1.5,0.2,0.5,0.1,0.7


##### 3 aylık çeyrek performanslar

Ocak - Şubat - Mart -> 1_4

Nisan - Mayıs - Haziran -> 2_4

Temmuz - Ağustos - Eylül -> 3_4

Ekim - Kasım - Aralık -> 4_4

In [12]:
df_2023_1_4 = df.loc[(df['Tarih'] >= '2023-01-01') & (df['Tarih'] <= '2023-03-31')]
df_2023_2_4 = df.loc[(df['Tarih'] >= '2023-04-01') & (df['Tarih'] <= '2023-06-30')]
df_2023_3_4 = df.loc[(df['Tarih'] >= '2023-07-01') & (df['Tarih'] <= '2023-09-30')]
df_2023_4_4 = df.loc[(df['Tarih'] >= '2023-10-01') & (df['Tarih'] <= '2023-12-31')]

df_2024_1_4 = df.loc[(df['Tarih'] >= '2024-01-01') & (df['Tarih'] <= '2024-03-31')]
df_2024_2_4 = df.loc[(df['Tarih'] >= '2024-04-01') & (df['Tarih'] <= '2024-06-30')]
df_2024_3_4 = df.loc[(df['Tarih'] >= '2024-07-01') & (df['Tarih'] <= '2024-09-30')]
df_2024_4_4 = df.loc[(df['Tarih'] >= '2024-10-01') & (df['Tarih'] <= '2024-12-31')]

df_2025_1_4 = df.loc[(df['Tarih'] >= '2025-01-01') & (df['Tarih'] <= '2025-03-31')]
df_2025_2_4 = df.loc[(df['Tarih'] >= '2025-04-01') & (df['Tarih'] <= '2025-06-30')]
df_2025_3_4 = df.loc[(df['Tarih'] >= '2025-07-01') & (df['Tarih'] <= '2025-09-30')]

In [13]:
df_2023_1_4 = df_2023_1_4.groupby(df_2023_1_4['Tarih'].dt.to_period("Y")).agg({'Yapılan': 'sum', 'Hedef': 'sum', 'FreeDay': 'sum', 'OffDay': 'sum', 'Yapilmayan': 'sum'})
df_2023_2_4 = df_2023_2_4.groupby(df_2023_2_4['Tarih'].dt.to_period("Y")).agg({'Yapılan': 'sum', 'Hedef': 'sum', 'FreeDay': 'sum', 'OffDay': 'sum', 'Yapilmayan': 'sum'})
df_2023_3_4 = df_2023_3_4.groupby(df_2023_3_4['Tarih'].dt.to_period("Y")).agg({'Yapılan': 'sum', 'Hedef': 'sum', 'FreeDay': 'sum', 'OffDay': 'sum', 'Yapilmayan': 'sum'})
df_2023_4_4 = df_2023_4_4.groupby(df_2023_4_4['Tarih'].dt.to_period("Y")).agg({'Yapılan': 'sum', 'Hedef': 'sum', 'FreeDay': 'sum', 'OffDay': 'sum', 'Yapilmayan': 'sum'})
df_2024_1_4 = df_2024_1_4.groupby(df_2024_1_4['Tarih'].dt.to_period("Y")).agg({'Yapılan': 'sum', 'Hedef': 'sum', 'FreeDay': 'sum', 'OffDay': 'sum', 'Yapilmayan': 'sum'})
df_2024_2_4 = df_2024_2_4.groupby(df_2024_2_4['Tarih'].dt.to_period("Y")).agg({'Yapılan': 'sum', 'Hedef': 'sum', 'FreeDay': 'sum', 'OffDay': 'sum', 'Yapilmayan': 'sum'})
df_2024_3_4 = df_2024_3_4.groupby(df_2024_3_4['Tarih'].dt.to_period("Y")).agg({'Yapılan': 'sum', 'Hedef': 'sum', 'FreeDay': 'sum', 'OffDay': 'sum', 'Yapilmayan': 'sum'})
df_2024_4_4 = df_2024_4_4.groupby(df_2024_4_4['Tarih'].dt.to_period("Y")).agg({'Yapılan': 'sum', 'Hedef': 'sum', 'FreeDay': 'sum', 'OffDay': 'sum', 'Yapilmayan': 'sum'})
df_2025_1_4 = df_2025_1_4.groupby(df_2025_1_4['Tarih'].dt.to_period("Y")).agg({'Yapılan': 'sum', 'Hedef': 'sum', 'FreeDay': 'sum', 'OffDay': 'sum', 'Yapilmayan': 'sum'})
df_2025_2_4 = df_2025_2_4.groupby(df_2025_2_4['Tarih'].dt.to_period("Y")).agg({'Yapılan': 'sum', 'Hedef': 'sum', 'FreeDay': 'sum', 'OffDay': 'sum', 'Yapilmayan': 'sum'})
df_2025_3_4 = df_2025_3_4.groupby(df_2025_3_4['Tarih'].dt.to_period("Y")).agg({'Yapılan': 'sum', 'Hedef': 'sum', 'FreeDay': 'sum', 'OffDay': 'sum', 'Yapilmayan': 'sum'})

df_2023_1_4['Donem'] = '2023_1_4'
df_2023_2_4['Donem'] = '2023_2_4'
df_2023_3_4['Donem'] = '2023_3_4'
df_2023_4_4['Donem'] = '2023_4_4'
df_2024_1_4['Donem'] = '2024_1_4'
df_2024_2_4['Donem'] = '2024_2_4'
df_2024_3_4['Donem'] = '2024_3_4'
df_2024_4_4['Donem'] = '2024_4_4'
df_2025_1_4['Donem'] = '2025_1_4'
df_2025_2_4['Donem'] = '2025_2_4'
df_2025_3_4['Donem'] = '2025_3_4'

df_2023_1_4['Calisma%'] = np.round( ((df_2023_1_4['Yapılan'] / df_2023_1_4['Hedef']) * 100) ,1)
df_2023_2_4['Calisma%'] = np.round( ((df_2023_2_4['Yapılan'] / df_2023_2_4['Hedef']) * 100) ,1)
df_2023_3_4['Calisma%'] = np.round( ((df_2023_3_4['Yapılan'] / df_2023_3_4['Hedef']) * 100) ,1)
df_2023_4_4['Calisma%'] = np.round( ((df_2023_4_4['Yapılan'] / df_2023_4_4['Hedef']) * 100) ,1)
df_2024_1_4['Calisma%'] = np.round( ((df_2024_1_4['Yapılan'] / df_2024_1_4['Hedef']) * 100) ,1)
df_2024_2_4['Calisma%'] = np.round( ((df_2024_2_4['Yapılan'] / df_2024_2_4['Hedef']) * 100) ,1)
df_2024_3_4['Calisma%'] = np.round( ((df_2024_3_4['Yapılan'] / df_2024_3_4['Hedef']) * 100) ,1)
df_2024_4_4['Calisma%'] = np.round( ((df_2024_4_4['Yapılan'] / df_2024_4_4['Hedef']) * 100) ,1)
df_2025_1_4['Calisma%'] = np.round( ((df_2025_1_4['Yapılan'] / df_2025_1_4['Hedef']) * 100) ,1)
df_2025_2_4['Calisma%'] = np.round( ((df_2025_2_4['Yapılan'] / df_2025_2_4['Hedef']) * 100) ,1)
df_2025_3_4['Calisma%'] = np.round( ((df_2025_3_4['Yapılan'] / df_2025_3_4['Hedef']) * 100) ,1)




In [14]:
df_2023_1_4.set_index('Donem', inplace=True)
df_2023_2_4.set_index('Donem', inplace=True)
df_2023_3_4.set_index('Donem', inplace=True)
df_2023_4_4.set_index('Donem', inplace=True)
df_2024_1_4.set_index('Donem', inplace=True)
df_2024_2_4.set_index('Donem', inplace=True)
df_2024_3_4.set_index('Donem', inplace=True)
df_2024_4_4.set_index('Donem', inplace=True)
df_2025_1_4.set_index('Donem', inplace=True)
df_2025_2_4.set_index('Donem', inplace=True)
df_2025_3_4.set_index('Donem', inplace=True)

In [15]:
merged_ceyrek = pd.concat([
    df_2023_1_4, df_2023_2_4, df_2023_3_4, df_2023_4_4,
    df_2024_1_4, df_2024_2_4, df_2024_3_4, df_2024_4_4,
    df_2025_1_4, df_2025_2_4, df_2025_3_4
])

In [16]:
merged_ceyrek

,Yapılan,Hedef,FreeDay,OffDay,Yapilmayan,Calisma%
Donem,,,,,,
2023_1_4,251,396,20,18,4,63.4
2023_2_4,319,618,10,15,4,51.6
2023_3_4,61,182,8,37,20,33.5
2023_4_4,200,394,25,8,8,50.8
2024_1_4,259,496,24,2,4,52.2
2024_2_4,251,480,24,7,2,52.3
2024_3_4,314,718,12,9,0,43.7
2024_4_4,258,670,25,4,4,38.5
2025_1_4,301,629,24,6,1,47.9


#### 5 - Haftalık Çalışmalar

In [17]:
df['Week'] = 0
add = 1
for i in range(0, len(df),7):
    df.loc[i:i+6, 'Week'] = add
    add += 1


In [18]:
haftalik_toplam = df.groupby(df['Week']).sum({'Yapılan': 'sum', 'Hedef': 'sum', 'FreeDay': 'sum', 'OffDay': 'sum', 'Yapilmayan': 'sum'})
haftalik_toplam['ÇalışmaOranı'] = (haftalik_toplam['Yapılan'] / haftalik_toplam['Hedef']*100).round(1)
haftalik_toplam.tail(3)

,Yapılan,Hedef,ÇalışmaOranı,FreeDay,OffDay,Yapilmayan
Week,,,,,,
146,11,38,28.9,2,0,1
147,15,44,34.1,2,0,0
148,5,16,31.2,0,0,0


// 2023, 2024 ve 2025 çalışmalarını kıyasla

#### 6 - Yıllık Çalışmalar

In [19]:
df_2022 = df.loc[(df['Tarih'] >= '2022-01-01') & (df['Tarih'] <= '2022-12-31')]
df_2023 = df.loc[(df['Tarih'] >= '2023-01-01') & (df['Tarih'] <= '2023-12-31')]
df_2024 = df.loc[(df['Tarih'] >= '2024-01-01') & (df['Tarih'] <= '2024-12-31')]
df_2025 = df.loc[(df['Tarih'] >= '2025-01-01') & (df['Tarih'] <= '2025-12-31')]

df_2022 = df_2022.groupby(df_2022['Tarih'].dt.to_period("Y")).agg({'Yapılan': 'sum', 'Hedef': 'sum', 'FreeDay': 'sum', 'OffDay': 'sum', 'Yapilmayan': 'sum'})
df_2023 = df_2023.groupby(df_2023['Tarih'].dt.to_period("Y")).agg({'Yapılan': 'sum', 'Hedef': 'sum', 'FreeDay': 'sum', 'OffDay': 'sum', 'Yapilmayan': 'sum'})
df_2024 = df_2024.groupby(df_2024['Tarih'].dt.to_period("Y")).agg({'Yapılan': 'sum', 'Hedef': 'sum', 'FreeDay': 'sum', 'OffDay': 'sum', 'Yapilmayan': 'sum'})
df_2025 = df_2025.groupby(df_2025['Tarih'].dt.to_period("Y")).agg({'Yapılan': 'sum', 'Hedef': 'sum', 'FreeDay': 'sum', 'OffDay': 'sum', 'Yapilmayan': 'sum'})

df_2022['Yıl'] = '2022'
df_2023['Yıl'] = '2023'
df_2024['Yıl'] = '2024'
df_2025['Yıl'] = '2025'

df_2022['BoşGünler'] = df_2022['FreeDay'] + df_2022['OffDay'] + df_2022['Yapilmayan']
df_2023['BoşGünler'] = df_2023['FreeDay'] + df_2023['OffDay'] + df_2023['Yapilmayan']
df_2024['BoşGünler'] = df_2024['FreeDay'] + df_2024['OffDay'] + df_2024['Yapilmayan']
df_2025['BoşGünler'] = df_2025['FreeDay'] + df_2025['OffDay'] + df_2025['Yapilmayan']

df_2022['Calisma%'] = np.round( ((df_2022['Yapılan'] / df_2022['Hedef']) * 100) ,1)
df_2023['Calisma%'] = np.round( ((df_2023['Yapılan'] / df_2023['Hedef']) * 100) ,1)
df_2024['Calisma%'] = np.round( ((df_2024['Yapılan'] / df_2024['Hedef']) * 100) ,1)
df_2025['Calisma%'] = np.round( ((df_2025['Yapılan'] / df_2025['Hedef']) * 100) ,1)

df_2022.set_index('Yıl', inplace=True)
df_2023.set_index('Yıl', inplace=True)
df_2024.set_index('Yıl', inplace=True)
df_2025.set_index('Yıl', inplace=True)

merged_yillik = pd.concat([df_2022, df_2023, df_2024, df_2025])
merged_yillik

,Yapılan,Hedef,FreeDay,OffDay,Yapilmayan,BoşGünler,Calisma%
Yıl,,,,,,,
2022,59,124,0,4,6,10,47.6
2023,831,1590,63,78,36,177,52.3
2024,1082,2364,85,22,10,117,45.8
2025,779,1779,66,19,10,95,43.8


#### 7 - Grafikler

##### Aylık Grafik

In [20]:
plot_data = aylik_toplam.copy()
plot_data['Tarih'] = plot_data['Tarih'].astype(str)

fig = px.line(plot_data, 
              x='Tarih', 
              y='ÇalışmaOranı',
              title='Aylara Göre Çalışma Oranları',
              labels={'ÇalışmaOranı': 'ÇalışmaOranı', 'Tarih': 'Tarih'},
              markers=True)
fig.update_layout(
    xaxis_tickangle=-45,
    showlegend=False
)

fig.show()

In [21]:
fig = px.line(plot_data, 
              x='Tarih', 
              y=['Yapılan','Hedef'],
              title='Aylık Yapılan Çalışmalar',
              labels={'Yapılan': 'Yapılan', 'Hedef': 'Hedef', 'Tarih': 'Tarih'},
              markers=True,
              color_discrete_sequence=["#0AE087","orange"]
              )
fig.update_layout(
    xaxis_tickangle=-45,
    showlegend=True
)

fig.show()

##### Haftalık Grafik

In [22]:
fig = px.line(haftalik_toplam.reset_index(), 
              x='Week', 
              y='ÇalışmaOranı',
              title='Haftalık Çalışma Oranları',
              labels={'ÇalışmaOranı': 'ÇalışmaOranı', 'Week': 'Week'},
              markers=True)
fig.update_layout(
    xaxis_tickangle=-45,
    showlegend=False
)

fig.show()

In [23]:
fig = px.line(haftalik_toplam.reset_index(), 
              x='Week', 
              y=['Yapılan','Hedef'],
              title='Haftalık Yapılan Çalışmalar',
              labels={'Yapılan': 'Yapılan', 'Hedef': 'Hedef', 'Week': 'Week'},
              markers=True,
              color_discrete_sequence=["#0AE087", "#FF4B4B"])
fig.update_layout(
    xaxis_tickangle=-45,
    showlegend=True
)

fig.show()

##### Yıllık Grafik

In [24]:
fig = px.line(merged_yillik.reset_index(), 
              x='Yıl', 
              y='Calisma%',
              title='Yıllık Çalışma Oranları',
              markers=True)
fig.update_layout(
    xaxis_tickangle=-45,
    showlegend=False
)

fig.show()

In [25]:
fig = px.line(merged_yillik.reset_index(), 
              x='Yıl', 
              y=['Yapılan','Hedef', "BoşGünler"],
              title='Yıllık Yapılan Çalışmalar',
              labels={'Yapılan': 'Yapılan', 'Hedef': 'Hedef', 'Yıl': 'Yıl'},
              markers=True,
              color_discrete_sequence=["#0AE087","orange", "#FF4B4B"])
fig.update_layout(
    xaxis_tickangle=-45,
    showlegend=True
)

fig.show()

#### 8 - Öne Çıkan İstatistikler

##### En Çok Çalışma Yapılan Ay

In [26]:
aylik_toplam[aylik_toplam['Yapılan'] == aylik_toplam['Yapılan'].max()]

,Tarih,Gün,Yapılan,Hedef,ÇalışmaOranı,FreeDay,OffDay,Yapilmayan,BoşGünler,Boş %
6,2023-06,30,127,216,58.8,0,8,0,8,26.7
19,2024-07,31,127,286,44.4,0,4,0,4,12.9


##### En Çok Hedef Belirlenen Ay

In [27]:
aylik_toplam[aylik_toplam['Hedef'] == aylik_toplam['Hedef'].max()]

,Tarih,Gün,Yapılan,Hedef,ÇalışmaOranı,FreeDay,OffDay,Yapilmayan,BoşGünler,Boş %
19,2024-07,31,127,286,44.4,0,4,0,4,12.9


##### En Çok Boş Geçen Ay

In [28]:
aylik_toplam[aylik_toplam['BoşGünler'] == aylik_toplam['BoşGünler'].max()]

,Tarih,Gün,Yapılan,Hedef,ÇalışmaOranı,FreeDay,OffDay,Yapilmayan,BoşGünler,Boş %
8,2023-08,31,9,50,18.0,2,16,9,27,87.1


##### En Az Boş Geçen Ay

In [29]:
aylik_toplam[aylik_toplam['BoşGünler'] == aylik_toplam['BoşGünler'].min()]

,Tarih,Gün,Yapılan,Hedef,ÇalışmaOranı,FreeDay,OffDay,Yapilmayan,BoşGünler,Boş %
19,2024-07,31,127,286,44.4,0,4,0,4,12.9


##### En Çok Çalışma Yapılan Hafta

In [30]:
haftalik_toplam[haftalik_toplam['Yapılan'] == haftalik_toplam['Yapılan'].max()]

,Yapılan,Hedef,ÇalışmaOranı,FreeDay,OffDay,Yapilmayan
Week,,,,,,
28,42,72,58.3,0,0,0


##### En çok Hedef Belirlenen Hafta

In [31]:
haftalik_toplam[haftalik_toplam['Hedef'] == haftalik_toplam['Hedef'].max()]

,Yapılan,Hedef,ÇalışmaOranı,FreeDay,OffDay,Yapilmayan
Week,,,,,,
102,33,80,41.2,0,0,0
